# Camada Bronze - Ingestão de Dados Brutos

Este notebook executa a ingestão de dados brutos de APIs governamentais e armazena na camada Bronze em formato Parquet.

## Estrutura da Camada Bronze
- `bronze/{fonte}/{dataset}/dt={data}/data.parquet`

## Fontes de Dados
- IBGE (municípios, estados, população)
- Portal da Transparência (BPC, órgãos SIAFI)

In [ ]:
# Instalar dependências se necessário
!pip install -q requests pandas minio pyarrow

In [ ]:
# Executar script de ingestão Bronze
exec(open('01_bronze_ingestion.py').read())

## Visualizar dados coletados

In [ ]:
import pandas as pd
from minio import Minio
import io

# Configurações
MINIO_SERVER_URL = "ch8ai-minio.l6zv5a.easypanel.host"
MINIO_ROOT_USER = "admin"
MINIO_ROOT_PASSWORD = "1q2w3e4r"
BUCKET_NAME = "govbr"

minio_client = Minio(
    MINIO_SERVER_URL,
    access_key=MINIO_ROOT_USER,
    secret_key=MINIO_ROOT_PASSWORD,
    secure=True
)

# Listar arquivos Bronze
objects = minio_client.list_objects(BUCKET_NAME, prefix="bronze/", recursive=True)
bronze_files = list(objects)

print(f"Total de arquivos na camada Bronze: {len(bronze_files)}")
for obj in bronze_files:
    print(f"📁 {obj.object_name} ({obj.size/1024:.2f} KB)")

In [ ]:
# Ler e visualizar um arquivo exemplo
if bronze_files:
    exemplo = bronze_files[0]
    response = minio_client.get_object(BUCKET_NAME, exemplo.object_name)
    df = pd.read_parquet(io.BytesIO(response.read()))
    response.close()
    response.release_conn()
    
    print(f"Arquivo: {exemplo.object_name}")
    print(f"Registros: {len(df)}")
    print(f"\nPrimeiras linhas:")
    print(df.head())
    print(f"\nColunas: {df.columns.tolist()}")